In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: longmeow. Use `wandb login --relogin` to force relogin


True

In [2]:
import os
import yaml
import sys
import time
from copy import deepcopy
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import math
from torch.autograd import Variable
from torch.nn.modules.normalization import LayerNorm


In [3]:
with open('/home/quanhhh/Documents/Improve-RUL-Prediction/bo.yml') as f:
  sweep_config = yaml.safe_load(f)

In [4]:
#model.py

import copy
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.modules.normalization import LayerNorm


class ConvLayer(nn.Module):
    """1-D Convolution layer to extract high-level features of each time-series input
    :param n_features: Number of input features/nodes (d_model)
    :param window_size: length of the input sequence
    :param kernel_size: size of kernel to use in the convolution operation
    """

    def __init__(self, n_features, kernel_size):
        super().__init__()
        self.padding = nn.ConstantPad1d((kernel_size - 1) // 2, 0.0)
        self.conv = nn.Conv1d(in_channels=n_features, out_channels=n_features, kernel_size=kernel_size)
        self.relu = nn.ReLU()
 
    def forward(self, src):
        src = src.permute(0, 2, 1)
        src = self.padding(src)
        src = self.relu(self.conv(src))
        return src.permute(0, 2, 1)  # Permute back


class TransformerModel(nn.Module):
    def __init__(self, encoder, src_embed, linear, conv):
        super().__init__()
        self.encoder = encoder
        self.src_embed = src_embed
        self.linear = linear
        self.conv = conv

    def forward(self, src, src_mask=None):
        src = self.conv(src)
        output = F.relu(self.linear(
            self.encoder(self.src_embed(src), src_mask)))
        return output


def clones(module, N):
    """
    Produce N identical layers
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class TransformerEncoder(nn.Module):
    "Core encoder is a stack of N layers"

    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn"
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        """
        Apply residual connection to any sublayer with the same size.
        """
        return x + self.dropout(sublayer(self.norm(x)))


class TransformerEncoderLayer(nn.Module):
    """
    Encoder is made up of self-attn and feed foward
    """

    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask=None):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)


def attention(query, key, value, device, mask=None, dropout=0.0):
    """Compute the Scaled Dot-Product Attention"""
    query = query.to(device)
    key = query.to(device)
    value = query.to(device)
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        mask = mask.to(device)
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)
    p_attn = F.dropout(p_attn, p=dropout)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadAttention(nn.Module):
    def __init__(self, h, d_model, device, dropout=0.1):
        """
        Takes in model size and number of heads.
        """
        super().__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.p = dropout
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.device = device

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linears, (query, key, value))]
        # 2) Apply attention on all the projected vectors in batch
        x, self.attn = attention(query, key, value, self.device, mask=mask, dropout=self.p)
        # 3) "Concat" using a view and apply a final linear
        x = x.transpose(1, 2).contiguous().view(
            nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)


class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        # Torch linears have a "b" by default.
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))


class PositionalEncoding(nn.Module):
    """ Implements the PE function. """

    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        try:
            pe[:, 1::2] = torch.cos(position * div_term)
        except:
            div_term = torch.exp(torch.arange(0, d_model - 1, 2)
                                 * (-math.log(10000.0) / d_model))
            pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1), :], requires_grad=False)
        return self.dropout(x)


class FNetHybridModel(nn.Module):
    def __init__(self, trans, src_embed, linear, fnet, conv):
        super().__init__()
        self.trans = trans
        self.src_embed = src_embed
        self.linear = linear
        self.fnet = fnet
        self.conv = conv 

    def forward(self, src, src_mask=None):
        src = self.conv(src)
        output = F.relu(self.linear(self.fnet(
            self.trans(self.src_embed(src), src_mask))))
        return output


class FNetEncoder(nn.Module):
    "Core Fnet is a stack of N layers"

    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x):
        "Pass the input through each layer in turn"
        for layer in self.layers:
            x = layer(x)
        return self.norm(x)



class FNetEncoderLayer(nn.Module):
    """
    Encoder is made up of a Fourier Mixing Layer and a FF Layer
    """

    def __init__(self, size, fft, feed_forward, dropout):
        super().__init__()
        self.fft = fft
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x):
        x = self.sublayer[0](x, lambda x: self.fft(x, x, x))
        return self.sublayer[1](x, self.feed_forward)


class FourierFFTLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, query, key, value):
        assert query is key
        assert key is value

        x = query
        return torch.real(torch.fft.fft(torch.fft.fft(x, dim=-1), dim=-2))


def create_transformer(N , d_model, l_win, device, kernel_size, d_ff=0, h=8, dropout=0.1):
    if (d_ff == 0):
        d_ff = d_model * 4
    c = copy.deepcopy
    conv = ConvLayer(d_model, kernel_size)
    attn = MultiHeadAttention(h, d_model, device, dropout)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout, l_win)
    final_linear = nn.Sequential(
        nn.Flatten(), nn.Dropout(dropout), nn.Linear(d_model * l_win, 1)
    )
    model = TransformerModel(
        TransformerEncoder(TransformerEncoderLayer(
            d_model, c(attn), c(ff), dropout), N),
        nn.Sequential(position),
        final_linear,
        conv
    )

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model


def create_fnet_hybrid(N, d_model, l_win, device, kernel_size, d_ff=0, h=8, dropout=0.1):
    if (d_ff == 0):
        d_ff = d_model * 4
    c = copy.deepcopy
    conv = ConvLayer(d_model, kernel_size)
    attn = MultiHeadAttention(h, d_model, device, dropout)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout, l_win)
    final_linear = nn.Sequential(
        nn.Flatten(), nn.Dropout(dropout), nn.Linear(d_model * l_win, 1)
    )
    fft = FourierFFTLayer()
    model = FNetHybridModel(
        TransformerEncoder(TransformerEncoderLayer(
            d_model, c(attn), c(ff), dropout), 1),
        nn.Sequential(position),
        final_linear,
        FNetEncoder(FNetEncoderLayer(d_model, c(fft), c(ff), dropout), N - 1), 
        conv
    )

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model



#dataloader.py
import numpy as np
import pandas as pd
from torch.utils.data import Dataset


class TimeSeriesDataset(Dataset):
    def __init__(self, config, mode):
        super().__init__()
        self.config = config
        self.mode = mode
        self.load_dataset(config)

    def __getitem__(self, idx):
        if self.mode == 'train':
            data = self.data[idx, :, :]
            label = self.label[idx]
            return data, label
        else:
            data = self.data[idx, :, :]
            label = self.label[idx]
            return data, label

    def __len__(self):
        return self.data.shape[0]

    def load_dataset(self, config):
        if self.mode == 'train':
            train_df = pd.read_csv("./preprocessed_data/train_004.csv")

            def gen_sequence(id_df, seq_length, seq_cols):
                data_array = id_df[seq_cols].values
                num_elements = data_array.shape[0]
                for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
                    yield data_array[start:stop, :]
            
            
            # #FD001
            # sensor_cols = [
            #     "s2",
            #     "s3",
            #     "s4",
            #     "s7",
            #     "s8",
            #     "s9",
            #     "s11",
            #     "s12",
            #     "s13",
            #     "s14",
            #     "s15",
            #     "s17",
            #     "s20",
            #     "s21",
            # ]
            # sequence_cols = ["setting1", "setting2"]

            # FD002

            # sensor_cols = [
            #     "s1",
            #     "s2",
            #     "s3",
            #     "s4",
            #     "s5",
            #     "s7",
            #     "s8",
            #     "s9",
            #     "s10",
            #     "s11",
            #     "s12",
            #     "s13",
            #     "s14",
            #     "s15",
            #     "s16",
            #     "s17",
            #     "s18",
            #     "s19",
            #     "s20",
            #     "s21",
            # ]
            # sequence_cols = ["setting1", "setting2", "setting3"]
            
            # #FD003
            # sensor_cols = [
            #     "s2",
            #     "s3",
            #     "s4",
            #     "s6",
            #     "s7",
            #     "s8",
            #     "s9",
            #     "s10",
            #     "s11",
            #     "s12",
            #     "s13",
            #     "s14",
            #     "s15",
            #     "s17",
            #     "s20",
            #     "s21",
            # ]
            # sequence_cols = ["setting1", "setting2"]

            # #FD004
  
            sensor_cols = [
                "s1",
                "s2",
                "s3",
                "s4",
                "s5",
                "s7",
                "s8",
                "s9",
                "s10",
                "s11",
                "s12",
                "s13",
                "s14",
                "s15",
                "s16",
                "s17",
                "s18",
                "s19",
                "s20",
                "s21",
            ]
            sequence_cols = ["setting1", "setting2", "setting3"]
            

            sequence_cols.extend(sensor_cols)
            # generator for the sequences
            seq_gen = (list(gen_sequence(train_df[train_df['id']==id], self.config['l_win'], sequence_cols)) 
                      for id in train_df['id'].unique())

            # generate sequences and convert to numpy array
            seq_array = np.concatenate(list(seq_gen)).astype(np.float32)

            # function to generate labels
            def gen_labels(id_df, seq_length, label):
                data_array = id_df[label].values
                num_elements = data_array.shape[0]
                return data_array[seq_length:num_elements, :]

            # generate labels
            label_gen = [gen_labels(train_df[train_df['id']==id], self.config['l_win'], ['RUL']) 
                        for id in train_df['id'].unique()]
            label_array = np.concatenate(label_gen).astype(np.float32)

            self.data = seq_array
            self.label = label_array

        else:
            test_df = pd.read_csv("./preprocessed_data/test_004.csv")
            
            #  #FD001

            # sensor_cols = [
            #     "s2",
            #     "s3",
            #     "s4",
            #     "s7",
            #     "s8",
            #     "s9",
            #     "s11",
            #     "s12",
            #     "s13",
            #     "s14",
            #     "s15",
            #     "s17",
            #     "s20",
            #     "s21",
            # ]
            # sequence_cols = ["setting1", "setting2"]

            #FD002

            # sensor_cols = [
            #     "s1",
            #     "s2",
            #     "s3",
            #     "s4",
            #     "s5",
            #     "s7",
            #     "s8",
            #     "s9",
            #     "s10",
            #     "s11",
            #     "s12",
            #     "s13",
            #     "s14",
            #     "s15",
            #     "s16",
            #     "s17",
            #     "s18",
            #     "s19",
            #     "s20",
            #     "s21",
            # ]
            # sequence_cols = ["setting1", "setting2", "setting3"]
            
            # # #FD003

            # sensor_cols = [
            #     "s2",
            #     "s3",
            #     "s4",
            #     "s6",
            #     "s7",
            #     "s8",
            #     "s9",
            #     "s10",
            #     "s11",
            #     "s12",
            #     "s13",
            #     "s14",
            #     "s15",
            #     "s17",
            #     "s20",
            #     "s21",
            # ]
            # sequence_cols = ["setting1", "setting2"]

            # #FD004

            sensor_cols = [
                "s1",
                "s2",
                "s3",
                "s4",
                "s5",
                "s7",
                "s8",
                "s9",
                "s10",
                "s11",
                "s12",
                "s13",
                "s14",
                "s15",
                "s16",
                "s17",
                "s18",
                "s19",
                "s20",
                "s21",
            ]
            sequence_cols = ["setting1", "setting2", "setting3"]
            
            sequence_cols.extend(sensor_cols)

            seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-config['l_win']:] 
                                  for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= config['l_win']]

            seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)


            y_mask = [len(test_df[test_df['id']==id]) >= config['l_win'] for id in test_df['id'].unique()]

            label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
            label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)

            self.data = seq_array_test_last
            self.label = label_array_test_last





#trainer.py
class ModelTrainer():
    def __init__(self, model, train_data, criterion, optimizer, device, config):
        self.model = model
        self.train_data = train_data
        self.device = device
        self.config = config
        self.train_loss_list = list()
        self.min_loss = float('inf')
        self.best_model = None
        self.best_optimizer = None
        self.optimizer = optimizer
        self.criterion = criterion

    def train_epoch(self, epoch):
        train_loss = 0.0
        self.model.train()
        for x, rul in self.train_data:
            self.model.zero_grad()
            out = self.model(x.to(self.device).float())
            loss = torch.sqrt(self.criterion(out.float(), rul.to(self.device).float()))
            loss.backward()
            self.optimizer.step()
            train_loss += loss
            

        train_loss = train_loss / len(self.train_data)
        wandb.log({"train loss": train_loss})
        self.train_loss_list.append(train_loss)

        if train_loss < self.min_loss:
            self.min_loss = train_loss
            self.best_model = deepcopy(self.model.state_dict())
            self.best_optimizer = deepcopy(self.optimizer.state_dict())
            self.best_epoch_in_round = epoch

    def train(self):
        self.model.to(self.device)

        for epoch in range(1, self.config['n_epochs'] + 1):
            self.train_epoch(epoch)
            wandb.log({"epoch": epoch})


        self.config['train_loss_list'] = self.train_loss_list

    def update_config(self):
        return self.config

In [5]:
#train.py
def training():
    with wandb.init(config=sweep_config):
        config = wandb.config

        torch.manual_seed(42)
        start = time.perf_counter()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        train_data = TimeSeriesDataset(config, mode='train')
        train_loader = DataLoader(train_data,
                                  batch_size=config['batch_size'],
                                  shuffle=True,
                                  num_workers=config['num_workers'])

        if config['model'] == 1:
            model = create_transformer(N=config['num_layers'],
                                            d_model=config['d_model'],
                                            l_win=config['l_win'],
                                            device=None,
                                            kernel_size=config['kernel_size'],
                                            d_ff=config['dff'],
                                            h=config['n_head'],
                                            dropout=config['dropout'])
        if config['model'] == 2:
            model = create_fnet_hybrid(N=config['num_layers'],
                                            d_model=config['d_model'],
                                            l_win=config['l_win'],
                                            device=None,
                                            kernel_size=config['kernel_size'],
                                            d_ff=config['dff'],
                                            h=config['n_head'],
                                            dropout=config['dropout'])

        optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], weight_decay=config['weight_decay'])
        criterion = nn.MSELoss()
        trainer = ModelTrainer(model, train_loader, criterion, optimizer, device, config)

        trainer.train()

        #inference.py

        test_data = TimeSeriesDataset(config, mode='test')
        test_loader = DataLoader(test_data,
                                    batch_size=1,
                                    shuffle=False,
                                    num_workers=config['num_workers'])

        model.to(device)
        test_loss = 0.0
        criterion = nn.MSELoss()
        test_loss_list = list()
        pred_list = list()
        with torch.no_grad():
            for x, rul in test_loader:
                out = model(x.to(device).float())
                loss = torch.sqrt(criterion(out.float(), rul.to(device).float()))
                test_loss += loss
                test_loss_list.append(loss)
                pred_list.append(out.float())

        test_loss_avg = test_loss / len(test_loader)
        truth_list = [rul.float().item() for x, rul in test_loader]
        training_time = (time.perf_counter() - start) / 60
        config['truth_list'] = truth_list
        config['pred_list'] = pred_list
        config['test_loss_avg'] = test_loss_avg
        config['test_loss_list_per_id'] = test_loss_list
        config['training_time'] = training_time
        wandb.log({"test_loss_avg": test_loss_avg})
        wandb.log({"training_time": training_time})
        if test_loss_avg < 12.4:
            wandb.alert(
                title='longmeow',
                text=f'test_loss_avg {test_loss_avg} is below the theshold 12.4',
            )
            print('Alert triggered')    

In [6]:
# sweep_id = wandb.sweep(sweep=sweep_config, project='improve-rul-prediction')

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.
wandb: WARNING   Violation 2. weight_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: 2nev6ie8
Sweep URL: https://wandb.ai/longmeow/improve-rul-prediction/sweeps/2nev6ie8


In [7]:
!date
wandb.agent('improve-rul-prediction/2nev6ie8', training, count=100000)

Mon Dec  5 10:07:58 AM +07 2022


wandb: Agent Starting Run: 4t0dyzvz with config:
wandb: 	batch_size: 64
wandb: 	d_model: 23
wandb: 	dff: 128
wandb: 	dropout: 0.1481999058765633
wandb: 	kernel_size: 5
wandb: 	l_win: 122
wandb: 	lr: 0.003997124489555377
wandb: 	model: 2
wandb: 	n_epochs: 89
wandb: 	n_head: 1
wandb: 	num_layers: 3
wandb: 	num_workers: 2
wandb: 	weight_decay: 0.006727809613743149
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
